In [ ]:
import threading
import socket
import sys
import os
def clientHandling(idConnection,addr,clientCount):
    
        try:
            filename = idConnection.recv(256).decode()
        except:
            print("#%s: Fallo al recibir el nombre del archivo"%(clientCount))
            idConnection.close()
            sys.exit();
        try:
            packages = idConnection.recv(16).decode()
        except:
            print("#%s: Fallo al recibir informacion del archivo"%(clientCount))
            idConnection.close()
            sys.exit();
        packages = int(packages)
       
        filename2 = ""
        for i in range(len(filename)):
            if filename[i] != '|':
                filename2 = filename2 + filename[i]
            else:
                break;
        filename = filename2
        
        print("#%s: %s es un archivo de %s paquetes" %(clientCount,filename,packages))
        dir = "../OUT/"+str(addr[0])+";"+str(addr[1])+"/"
        try:
            os.stat("../OUT")
        except:
            os.mkdir("../OUT")
        try:
            os.stat(dir)
        except:
            os.mkdir(dir)
            
        out_file = open(dir+"/"+filename, "wb") # open for [w]riting as [b]inary
        
        print("#%s: Recibiendo paquetes"%(clientCount))
        for i in range(packages):
            try:
                buffer = idConnection.recv(256)
                print("#%s: %s/%s"%(clientCount,i+1, packages))
            except:
                print("#%s: Fallo al recibir un paquete de datos: %s/%s"%(clientCount,i+1, packages))
                break;
            try:
                out_file.write(buffer)
            except:
                print("#%s: Fallo al escribir un paquete de datos: %s/%s"%(clientCount,i+1, packages))
                break;
        else:
            print("#%s: Se ha recibido el archivo correctamente"%(clientCount))
        
        out_file.close()
        idConnection.close()
        print("#%s: Conexión finalizada"%(clientCount))
        
if __name__ == "__main__":
    idSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM, proto=0)
    print("Socket creado exitosamente")

    host = "127.0.0.1"
    port = 12346

    idSocket.bind((host, port))
    print("Socket unido a %s:%s" % (host, port))


    idSocket.listen(1)
    print("Socket escuchando")
    
    clientCount = 1
    while True:
        idConnection, addr = idSocket.accept()
        print("Conexión #%s con %s:%s"%(clientCount,addr[0],addr[1]))
        idHilo = threading.Thread(target=clientHandling, args=[idConnection,addr,clientCount])
        idHilo.start()  
        clientCount = clientCount + 1
